In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc

Read in adata

In [ ]:
adata = sc.read("/mnt/sata2/Analysis_Alex_2/perturb1/final_filtered_on_leiden.h5ad")

In [ ]:
import scanpy as sc
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.gridspec import GridSpecFromSubplotSpec
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib.patches import Rectangle

# import mpl_scatter_density # adds projection='scatter_density'
import numpy as np
import warnings
import seaborn as sns
import math

warnings.filterwarnings("ignore")

In [ ]:
import matplotlib.colors as clr

zissou = [
    "#3A9AB2",
    "#6FB2C1",
    "#91BAB6",
    "#A5C2A3",
    "#BDC881",
    "#DCCB4E",
    "#E3B710",
    "#E79805",
    "#EC7A05",
    "#EF5703",
    "#F11B00",
]
spectral = [
    "#A71B4BFF",
    "#B72E48FF",
    "#C63F41FF",
    "#D54F34FF",
    "#E35F15FF",
    "#E97302FF",
    "#EE850EFF",
    "#F29723FF",
    "#F5A838FF",
    "#F8B84EFF",
    "#FAC763FF",
    "#FBD678FF",
    "#FDE38DFF",
    "#FEEFA2FF",
    "#FEF9B5FF",
    "#F4FCBAFF",
    "#DEF7B4FF",
    "#C4F1AFFF",
    "#A9E9ADFF",
    "#8BE1ADFF",
    "#6BD7AFFF",
    "#47CCB1FF",
    "#0BC0B3FF",
    "#00B4B5FF",
    "#00A6B6FF",
    "#0096B5FF",
    "#0086B3FF",
    "#2574AFFF",
    "#4461A8FF",
    "#584B9FFF",
]


colormap = clr.LinearSegmentedColormap.from_list("Zissou", zissou)
colormap_r = clr.LinearSegmentedColormap.from_list("Zissou", zissou[::-1])

In [ ]:
# Functions to help with the IMAPS


# https://stackoverflow.com/questions/20105364/how-can-i-make-a-scatter-plot-colored-by-density/53865762#53865762
def scatter_with_gaussian_kde(ax, x, y, weights=None, **kwargs):
    # https://stackoverflow.com/a/20107592/3015186
    # Answer by Joel Kington
    from scipy.stats import gaussian_kde

    xy = np.vstack([x, y])

    try:
        z = gaussian_kde(xy, weights=weights)(xy)
    except:
        z = gaussian_kde(xy)(xy)

    ax.scatter(x, y, c=z, **kwargs)


# Custom biexponential transformation. Maybe not needed for IF data
def transformation(x, a=0.1, b=0.1, c=0.5, d=2.5, f=4, w=1):
    x = np.array(x)
    return a * np.exp(b * ((x - w))) - c * np.exp(-d * (x - w)) + f

In [ ]:
# Coordinates of the gates
gates = {
    "Top IE": {
        "edges": [
            [0, 0.45],
            [0.34, 0.45],
            [0.34, 1.03],
            [0, 1.03],
        ],
        "label_position": {"x": 0.16, "y": 0.9},
        "fill": "#3A9AB244",
    },
    "Top LP": {
        "edges": [
            [0.34, 0.45],
            [1, 0.45],
            [1, 1.03],
            [0.34, 1.03],
        ],
        "label_position": {"x": 0.6, "y": 0.9},
        "fill": "#3A9AB244",
    },
    "Crypt LP": {
        "edges": [
            [0.34, 0.44],
            [1, 0.44],
            [0.34, 0.05],
        ],
        "label_position": {"x": 0.6, "y": 0.1},
        "fill": "#F11B0044",
    },
    "Crypt IE": {
        "edges": [
            [0, 0.44],
            [0.34, 0.44],
            [0.34, 0.05],
            [0.3, 0],
            [0, 0],
        ],
        "label_position": {"x": 0.16, "y": 0.05},
        "fill": "#F11B0044",
    },
    "Muscularis": {
        "edges": [[0.3, 0], [0.34, 0.05], [1, 0.44], [6, 0.44], [6, 0]],
        "label_position": {"x": 0.6, "y": 0.05},
        "fill": "#BDC88155",
    },
}


def draw_gates(ax, gates, transformation, type="edge"):
    from matplotlib.patches import Polygon

    for gate in gates:
        # Apply transformation to x values
        points = [
            [transformation(element[0])] + element[1:]
            for element in gates[gate]["edges"]
        ]

        if type == "fill":
            p = Polygon(points, facecolor=gates[gate]["fill"], edgecolor="none")
            ax.add_patch(p)
        elif type == "edge":
            p = Polygon(points, facecolor="none", edgecolor="#222222")
            ax.add_patch(p)

            ax.text(
                transformation(gates[gate]["label_position"]["x"]),
                gates[gate]["label_position"]["y"],
                gate,
                fontsize=6,
                color="#222222",
            )


def classify_cells(adata, gates, transformation=transformation):
    """
    Classify cells based on the gates.
    """
    from shapely.geometry import Point
    from shapely.geometry.polygon import Polygon
    import geopandas as gpd

    adata.obs["epithelial_distance_transformed"] = transformation(
        adata.obs["epithelial_distance"]
    )
    adata.obs["gate"] = False

    print("Creating polygons")
    polygons = {}
    for gate in gates:
        # Apply transformation to x values
        points = [
            [transformation(element[0])] + element[1:]
            for element in gates[gate]["edges"]
        ]
        polygons[gate] = Polygon(points)
    polygons = gpd.GeoSeries(polygons)
    gpd_poly = gpd.GeoDataFrame({"gates": polygons}, geometry="gates")

    print("Creating cells")
    cells = gpd.GeoSeries.from_xy(
        adata.obs["epithelial_distance_transformed"], adata.obs["crypt_villi_axis"]
    )
    gpd_cells = gpd.GeoDataFrame({"cells": cells}, geometry="cells")

    print("Joining cells and polygons")
    result = gpd.sjoin(
        gpd_cells,
        gpd_poly,
        how="left",
    )
    return result

In [ ]:
batches = {
    "SI_3": {"x": 2400, "y": 2400},
    "SI_2": {"x": 2400, "y": 2400},
}

In [ ]:
adata.obs["epithelial_distance_transformed"] = transformation(
    adata.obs["epithelial_distance"]
)

Display IMAPs for each perturbation group

In [ ]:
import numpy as np


# Create subplots
def plot_imaps(
    adata,
    batches,
    obs,
    value,
    ax_ticks=[0.15, 0.3, 0.6, 1, 6],
    transformation=transformation,
    gates=gates,
    gene=None,
    save_path=None,
):
    fig = plt.figure(figsize=(3, 3 * len(batches)), dpi=600)

    # Apply transformation

    # Iterate over batches
    for i, bt in enumerate(batches):
        sub_adata = adata[adata.obs["batch"] == bt]
        sub_adata = sub_adata[sub_adata.obs[obs] == value]

        classification = classify_cells(sub_adata, gates)

        ax = fig.add_subplot(len(batches), 1, i + 1)

        # Draw gates filled in background
        draw_gates(ax, gates=gates, transformation=transformation, type="fill")

        # Draw the density lines
        if gene == None:
            sns.kdeplot(
                data=sub_adata.obs,
                x="epithelial_distance_transformed",
                y="crypt_villi_axis",
                ax=ax,
                color="#444444",
                linewidths=0.5,
                cmap="viridis",
            )

            # Colored scatter plot
            scatter_with_gaussian_kde(
                ax=ax,
                x=sub_adata.obs["epithelial_distance_transformed"],
                y=sub_adata.obs["crypt_villi_axis"],
                s=5,
                cmap="viridis",
            )

        else:
            sns.kdeplot(
                data=sub_adata.obs,
                x="epithelial_distance_transformed",
                y="crypt_villi_axis",
                ax=ax,
                color="#444444",
                linewidths=0.5,
                weights=sub_adata.X[:, sub_adata.var.index.isin([gene])].flatten(),
                cmap="viridis",
            )

            # Colored scatter plot
            scatter_with_gaussian_kde(
                ax=ax,
                x=sub_adata.obs["epithelial_distance_transformed"],
                y=sub_adata.obs["crypt_villi_axis"],
                s=5,
                weights=sub_adata.X[:, sub_adata.var.index.isin([gene])].flatten(),
                cmap="viridis",
            )

        value_counts = classification["index_right"].value_counts()
        norm_counts = value_counts / np.sum(value_counts)
        all_titles = ""
        for index, value_ in zip(norm_counts.index.values, norm_counts.values):
            # Get the unnormalized count
            unnormalized_count = value_counts[index]
            # Construct the title
            all_titles += f"{index}: Norm Count - {value_}, Unnorm Count - {unnormalized_count}, \n"

        file_path = f"figures/panel_c_{value}.txt"  # You can change the file name and path as needed

        # Open the file in write mode
        with open(file_path, "w") as file:
            # Write the titles to the file
            file.write(all_titles)
        # Transform the tick labels and set them
        ax.set_xticks(transformation(ax_ticks))
        ax.set_xticklabels(ax_ticks)

        # Label the axes
        ax.set_xlabel("Epithelial Axis")
        ax.set_ylabel("Crypt-Villi Axis")

        ax.set_ylim(-0.02, 1.05)
        ax.grid(False)
        # Add a title
        ax.set_title(f"{value}_{bt}")
        draw_gates(ax, gates=gates, transformation=transformation)

    fig.tight_layout()
    try:
        os.mkdir(save_path)
    except:
        pass

    try:
        fig.savefig(os.path.join(save_path, f"panel_c_imaps_{value}.pdf"))
    except:
        fig.savefig(os.path.join(save_path, f"panel_c_imaps_{value}_density.pdf"))
    plt.show()
    plt.close()


# plot_imaps(adata, batches, obs='Subtype', value='Cd8_T-Cell_P14')
plot_imaps(adata, batches, obs="guide_rnas", value="sgCd19", save_path="figures/imaps")
plot_imaps(adata, batches, obs="guide_rnas", value="sgThy1", save_path="figures/imaps")
plot_imaps(adata, batches, obs="guide_rnas", value="sgCxcr3", save_path="figures/imaps")

In [ ]:
def classify_cells(adata, gates, transformation=transformation):
    """
    Classify cells based on the IMAP gates.

    Parameters:
    - adata (anndata): The anndata object containing the cells to classify.
    - gates (dict): A dictionary containing the gates to classify the cells with.
    - transformation (function): A function to transform the x values of the gates.

    Returns:
    - result (geopandas dataframe): A geopandas dataframe containing the classified cells.
    """
    from shapely.geometry import Point
    from shapely.geometry.polygon import Polygon
    import geopandas as gpd

    adata.obs["epithelial_distance_transformed"] = transformation(
        adata.obs["epithelial_distance"]
    )
    adata.obs["gate"] = False

    print("Creating polygons")
    polygons = {}
    for gate in gates:
        # Apply transformation to x values
        points = [
            [transformation(element[0])] + element[1:]
            for element in gates[gate]["edges"]
        ]
        polygons[gate] = Polygon(points)
    polygons = gpd.GeoSeries(polygons)
    gpd_poly = gpd.GeoDataFrame({"gates": polygons}, geometry="gates")

    print("Creating cells")
    cells = gpd.GeoSeries.from_xy(
        adata.obs["epithelial_distance_transformed"], adata.obs["crypt_villi_axis"]
    )
    gpd_cells = gpd.GeoDataFrame({"cells": cells}, geometry="cells")

    print("Joining cells and polygons")
    result = gpd.sjoin(
        gpd_cells,
        gpd_poly,
        how="left",
    )
    return result


def get_mean_expression(adata, genes):
    """
    Calculate the mean expression of specified genes for each batch and gate in the provided AnnData object.

    Parameters:
    - adata (AnnData): Annotated data matrix with observations (rows) and variables (columns).
    - genes (list): A list of gene names for which mean expression is calculated.

    Returns:
    - pd.DataFrame: A DataFrame containing mean expression values for each gene, batch, and gate.
    """
    import re

    keys = genes + ["batch", "gate"]
    df = sc.get.obs_df(adata, keys=keys)
    grouped = (
        df.groupby(["batch", "gate"])
        .mean()
        .join(df.groupby(["batch", "gate"]).size().rename("group_size"))
        .reset_index()
    )
    grouped["Day"] = [re.findall(r"\d+", b)[0] for b in grouped["batch"]]

    # Melt into a longer form
    grouped = pd.melt(
        grouped,
        id_vars=["batch", "Day", "gate", "group_size"],
        var_name="gene",
        value_name="expression",
    )
    return grouped


def get_scaled_mean_expression(adata, genes):
    """
    Calculate the scaled mean expression of specified genes for each batch and gate in the provided AnnData object.

    Parameters:
    - adata (AnnData): Annotated data matrix with observations (rows) and variables (columns).
    - genes (list): A list of gene names for which scaled mean expression is calculated.

    Returns:
    - pd.DataFrame: A DataFrame containing scaled mean expression values for each gene, batch, and gate.
    """
    grouped = get_mean_expression(adata=adata, genes=genes)
    # Group the DataFrame by gene
    gene_groups = grouped.groupby("gene")

    # Apply the scaling function to each group
    def scale_group(group):
        gene_min = group["expression"].min()
        gene_max = group["expression"].max()
        group["expression"] = (group["expression"] - gene_min) / (gene_max - gene_min)
        return group

    df_scaled = gene_groups.apply(scale_group)

    return df_scaled

In [ ]:
classification = classify_cells(adata, gates)
adata.obs["gate"] = classification["index_right"]

In [ ]:
adata = adata[adata.obs["guide_rnas"].isin(["sgCd19", "sgCxcr3", "sgThy1"])]

In [ ]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import sem, ttest_ind
import itertools
from statsmodels.stats.multitest import multipletests


def plot_gene_across_gates(adata, gene, savdir):
    try:
        os.mkdir(os.path.join(savdir, gene))
    except FileExistsError:
        print("Directory already exists")

    gate_order = ["Muscularis", "Crypt LP", "Crypt IE", "Top LP", "Top IE"]

    gate = adata.obs["gate"].values
    groups = adata.obs["guide_rnas"].values
    gene_expression = adata.X[:, adata.var.index.isin([gene])].flatten()

    # Create a DataFrame from the data
    data = pd.DataFrame(
        {"Gate": gate, "Group": groups, f"{gene} Expression": gene_expression}
    )

    # Ensure that 'Gate' is treated as a categorical variable with the specified order
    data["Gate"] = pd.Categorical(data["Gate"], categories=gate_order, ordered=True)

    # Group by 'Gate' and 'Group' and calculate the mean and SEM expression
    mean_expression = (
        data.groupby(["Gate", "Group"])[f"{gene} Expression"].mean().unstack()
    )
    sem_expression = (
        data.groupby(["Gate", "Group"])[f"{gene} Expression"].apply(sem).unstack()
    )

    # Calculate p-values between pairs of groups within each gate
    p_values = {}
    for gate_name, group_data in data.groupby("Gate"):
        group_combinations = list(
            itertools.combinations(group_data["Group"].unique(), 2)
        )
        p_values[gate_name] = {}
        for group1, group2 in group_combinations:
            group1_data = group_data[group_data["Group"] == group1][
                f"{gene} Expression"
            ]
            group2_data = group_data[group_data["Group"] == group2][
                f"{gene} Expression"
            ]
            t_stat, p_val = ttest_ind(group1_data, group2_data)
            p_values[gate_name][(group1, group2)] = p_val

    # Correct for multiple testing
    corrected_p_values = {}
    for gate_name in p_values:
        p_vals = list(p_values[gate_name].values())
        _, corrected_p_vals, _, _ = multipletests(p_vals, method="fdr_bh")
        corrected_p_values[gate_name] = dict(
            zip(p_values[gate_name].keys(), corrected_p_vals)
        )

    # Plot the bar plot with SEM error bars
    ax = mean_expression.plot(
        kind="bar", yerr=sem_expression, capsize=4, figsize=(10, 7), alpha=0.7
    )

    # Add significance markers
    for gate_idx, gate_name in enumerate(mean_expression.index):
        y_max = (
            mean_expression.loc[gate_name].max() + sem_expression.loc[gate_name].max()
        )
        y_min = mean_expression.loc[gate_name].min()
        for (group1, group2), p_val in corrected_p_values[gate_name].items():
            if p_val < 0.05:
                x1 = (
                    gate_idx
                    - 0.2
                    + list(mean_expression.columns).index(group1)
                    * 0.4
                    / len(mean_expression.columns)
                )
                x2 = (
                    gate_idx
                    - 0.2
                    + list(mean_expression.columns).index(group2)
                    * 0.4
                    / len(mean_expression.columns)
                )
                y = y_max + 0.05 * (y_max - y_min)
                ax.plot(
                    [x1, x1, x2, x2],
                    [y, y + 0.02 * y_max, y + 0.02 * y_max, y],
                    lw=1.5,
                    c="black",
                )
                ax.text(
                    (x1 + x2) / 2,
                    y + 0.03 * y_max,
                    np.round(p_val, 4),
                    ha="center",
                    va="bottom",
                    color="black",
                )
                y_max += 0.1 * (y_max - y_min)  # Increment for the next annotation

    plt.title(f"Mean {gene} Expression Across Gates and Groups")
    plt.ylabel(f"Mean {gene} Expression")
    plt.xlabel("Gate")
    plt.xticks(rotation=0)
    plt.legend(title="Group")
    plt.savefig(
        os.path.join(savdir, gene, f"{gene}_expression_across_gates_groups.pdf")
    )
    plt.show()

In [ ]:
savedir = "figures/gene_expression_across_gates"

try:
    os.mkdir(savedir)
except:
    print("Directory already exists")

Display gene expression within each perturbation group within each gate

In [ ]:
for gene in [
    "Fer1l6",
    "Neurog3",
    "Muc5ac",
    "Itgae",
    "Klrg1",
    "Cxcr3",
    "Thy1",
    "Gzma",
    "Gzmb",
    "Tcf7",
    "Il7r",
    "Il18r1",
    "Klf2",
]:
    plot_gene_across_gates(adata, gene, savedir)

Fraction of cells in each gate

In [ ]:
gates = adata.obs["gate"].values
sgRNAs = adata.obs["guide_rnas"].values

In [ ]:
# Define the desired gate order
gate_order = ["Muscularis", "Crypt LP", "Crypt IE", "Top LP", "Top IE"]

# Create a DataFrame to easily count occurrences
data = pd.DataFrame({"Gate": gates, "sgRNA": sgRNAs})

# Get the count of each sgRNA in each gate
pivot_table = data.groupby(["sgRNA", "Gate"]).size().unstack(fill_value=0)

# Reorder the columns according to gate_order
pivot_table = pivot_table[gate_order]

# Normalize to get proportions
pivot_table = pivot_table.div(pivot_table.sum(axis=1), axis=0)

# Plotting
fig, ax = plt.subplots(figsize=(10, 7), dpi=400)
bars = pivot_table.plot(kind="bar", stacked=True, ax=ax, colormap="tab20")

# Adding labels and title
ax.set_xlabel("sgRNA")
ax.set_ylabel("Proportion within each gate")
ax.set_title("Distribution of each guide RNA across gates")

# Adding annotations for each bar segment
for container in ax.containers:
    for i, bar in enumerate(container):
        height = bar.get_height()
        if height > 0:  # Only add annotation if the height is greater than 0
            ax.annotate(
                f"{height:.1%}",  # Format as percentage
                xy=(bar.get_x() + bar.get_width() / 2, bar.get_y() + height / 2),
                ha="center",
                va="center",
                fontsize=10,
                color="black",
            )

# Display the legend
ax.legend(title="Gate", bbox_to_anchor=(1.05, 1), loc="upper left")

# Save the plot
plt.savefig(
    "figures/gene_expression_across_gates/sgRNA_gate_distribution_comparison.pdf",
    bbox_inches="tight",
)

# Show the plot
plt.show()

Barcode expression

In [ ]:
def plot_gene_across_gates_raw(adata, gene, guide_rna, color, savdir):
    try:
        os.mkdir(os.path.join(savdir, gene))
    except FileExistsError:
        print("Directory already exists")

    gate_order = ["Muscularis", "Crypt LP", "Crypt IE", "Top LP", "Top IE"]

    # Filter for the specific guide RNA
    data = adata.obs.copy()
    data["Gene Expression"] = adata.layers["raw"][
        :, adata.var.index.isin([gene])
    ].flatten()
    data = data[data["guide_rnas"] == guide_rna]

    # Ensure 'Gate' is treated as a categorical variable with the specified order
    data["gate"] = pd.Categorical(data["gate"], categories=gate_order, ordered=True)

    # Group by 'Gate' and calculate the mean and SEM expression
    mean_expression = data.groupby("gate")["Gene Expression"].mean()
    sem_expression = data.groupby("gate")["Gene Expression"].apply(sem)

    # Perform statistical tests across gates for the specific guide RNA
    p_values = {}
    gate_combinations = list(itertools.combinations(gate_order, 2))
    for gate1, gate2 in gate_combinations:
        gate1_data = data[data["gate"] == gate1]["Gene Expression"]
        gate2_data = data[data["gate"] == gate2]["Gene Expression"]
        t_stat, p_val = ttest_ind(
            gate1_data, gate2_data, equal_var=False
        )  # Using Welch's t-test for unequal variance
        p_values[(gate1, gate2)] = p_val

    # Correct for multiple testing
    corrected_p_values = {}
    p_vals = list(p_values.values())
    _, corrected_p_vals, _, _ = multipletests(p_vals, method="fdr_bh")
    corrected_p_values = dict(zip(p_values.keys(), corrected_p_vals))

    # Plot the bar plot with SEM error bars
    fig, ax = plt.subplots(figsize=(10, 7))
    ax.bar(
        mean_expression.index,
        mean_expression,
        yerr=sem_expression,
        capsize=4,
        alpha=0.7,
        color=color,
    )

    # Add significance markers
    y_max = mean_expression.max() + sem_expression.max()
    y_min = mean_expression.min()
    for (gate1, gate2), p_val in corrected_p_values.items():
        if p_val < 0.05:
            x1 = gate_order.index(gate1)
            x2 = gate_order.index(gate2)
            y = y_max + 0.05 * (y_max - y_min)
            ax.plot(
                [x1, x1, x2, x2],
                [y, y + 0.02 * y_max, y + 0.02 * y_max, y],
                lw=1.5,
                c="black",
            )
            ax.text(
                (x1 + x2) / 2,
                y + 0.03 * y_max,
                f"p={np.round(p_val, 4)}",
                ha="center",
                va="bottom",
                color="black",
            )
            y_max += 0.1 * (y_max - y_min)  # Increment for the next annotation

    plt.title(f"{gene} Expression Across Gates for {guide_rna}")
    plt.ylabel(f"{gene} Expression")
    plt.xlabel("Gate")
    plt.xticks(rotation=45)
    plt.savefig(
        os.path.join(
            savdir, gene, f"{gene}_expression_across_gates_for_{guide_rna}_raw.pdf"
        )
    )
    plt.show()

In [ ]:
# for gene in [, 'Neurog3', 'Muc5ac']:
savedir = "figures/gene_expression_across_gates"
plot_gene_across_gates_raw(adata, "Fer1l6", "sgCxcr3", color="orange", savdir=savedir)
plot_gene_across_gates_raw(
    adata, "Neurog3", "sgThy1", color="lightgreen", savdir=savedir
)
plot_gene_across_gates_raw(adata, "Muc5ac", "sgCd19", color="lightblue", savdir=savedir)

Expression in positive cells


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import sem, ttest_ind
import itertools
from statsmodels.stats.multitest import multipletests


def plot_gene_across_gates_positive(adata, gene_postive, gene_plot, savdir):

    arr_express = np.array(
        adata.layers["raw"][:, adata.var.index.isin([gene_postive])]
    ).flatten()
    where_pos = np.where(arr_express > 0)[0]
    adata.obs["gene_postive"] = False
    adata.obs["gene_postive"][where_pos] = True
    adata = adata[where_pos].copy()
    try:
        os.mkdir(os.path.join(savdir, gene_postive + "_colored_by_" + gene_plot))
    except FileExistsError:
        print("Directory already exists")

    gate_order = ["Muscularis", "Crypt LP", "Crypt IE", "Top LP", "Top IE"]

    gate = adata.obs["gate"].values
    groups = adata.obs["guide_rnas"].values
    gene_expression = adata.X[:, adata.var.index.isin([gene_plot])].flatten()

    # Create a DataFrame from the data
    data = pd.DataFrame(
        {"Gate": gate, "Group": groups, f"{gene} Expression": gene_expression}
    )

    # Ensure that 'Gate' is treated as a categorical variable with the specified order
    data["Gate"] = pd.Categorical(data["Gate"], categories=gate_order, ordered=True)

    # Group by 'Gate' and 'Group' and calculate the mean and SEM expression
    mean_expression = (
        data.groupby(["Gate", "Group"])[f"{gene} Expression"].mean().unstack()
    )
    sem_expression = (
        data.groupby(["Gate", "Group"])[f"{gene} Expression"].apply(sem).unstack()
    )

    # Calculate p-values between pairs of groups within each gate
    p_values = {}
    for gate_name, group_data in data.groupby("Gate"):
        group_combinations = list(
            itertools.combinations(group_data["Group"].unique(), 2)
        )
        p_values[gate_name] = {}
        for group1, group2 in group_combinations:
            group1_data = group_data[group_data["Group"] == group1][
                f"{gene} Expression"
            ]
            group2_data = group_data[group_data["Group"] == group2][
                f"{gene} Expression"
            ]
            t_stat, p_val = ttest_ind(group1_data, group2_data)
            p_values[gate_name][(group1, group2)] = p_val

    # Correct for multiple testing
    corrected_p_values = {}
    for gate_name in p_values:
        p_vals = list(p_values[gate_name].values())
        _, corrected_p_vals, _, _ = multipletests(p_vals, method="fdr_bh")
        corrected_p_values[gate_name] = dict(
            zip(p_values[gate_name].keys(), corrected_p_vals)
        )

    # Plot the bar plot with SEM error bars
    ax = mean_expression.plot(
        kind="bar", yerr=sem_expression, capsize=4, figsize=(10, 7), alpha=0.7
    )

    # Add significance markers
    for gate_idx, gate_name in enumerate(mean_expression.index):
        y_max = (
            mean_expression.loc[gate_name].max() + sem_expression.loc[gate_name].max()
        )
        y_min = mean_expression.loc[gate_name].min()
        for (group1, group2), p_val in corrected_p_values[gate_name].items():
            if p_val < 0.05:
                x1 = (
                    gate_idx
                    - 0.2
                    + list(mean_expression.columns).index(group1)
                    * 0.4
                    / len(mean_expression.columns)
                )
                x2 = (
                    gate_idx
                    - 0.2
                    + list(mean_expression.columns).index(group2)
                    * 0.4
                    / len(mean_expression.columns)
                )
                y = y_max + 0.05 * (y_max - y_min)
                ax.plot(
                    [x1, x1, x2, x2],
                    [y, y + 0.02 * y_max, y + 0.02 * y_max, y],
                    lw=1.5,
                    c="black",
                )
                ax.text(
                    (x1 + x2) / 2,
                    y + 0.03 * y_max,
                    np.round(p_val, 4),
                    ha="center",
                    va="bottom",
                    color="black",
                )
                y_max += 0.1 * (y_max - y_min)  # Increment for the next annotation

    plt.title(
        f"Mean {gene_plot} Expression Across Gates and Groups in {gene_postive} Positive Cells"
    )
    plt.ylabel(f"Mean {gene_plot} Expression")
    plt.xlabel("Gate")
    plt.xticks(rotation=0)
    plt.legend(title="Group")
    # plt.savefig(os.path.join(savdir, gene_postive + '_colored_by_' + gene_plot, f'{gene_postive}_colored_by_{gene_plot}_across_gates_groups.pdf'))
    plt.show()

In [ ]:
plot_gene_across_gates_positive(adata, "Itgae", "Klf2", savedir)

In [ ]:
import os
import numpy as np
import pandas as pd
import itertools
from scipy.stats import sem, ttest_ind
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt


def plot_fraction_across_gates_positive(adata, gene_postive, gene_plot, savdir):
    arr_express = np.array(
        adata.layers["raw"][:, adata.var.index.isin([gene_postive])]
    ).flatten()
    where_pos = np.where(arr_express > 0)[0]
    adata.obs["gene_postive"] = False
    adata.obs["gene_postive"].iloc[where_pos] = True

    try:
        os.mkdir(os.path.join(savdir, gene_postive + "_colored_by_" + gene_plot))
    except FileExistsError:
        print("Directory already exists")

    gate_order = ["Muscularis", "Crypt LP", "Crypt IE", "Top LP", "Top IE"]

    gate = adata.obs["gate"].values
    groups = adata.obs["guide_rnas"].values
    positives = adata.obs["gene_postive"].values
    gene_expression = adata.X[:, adata.var.index.isin([gene_plot])].flatten()

    # Create a DataFrame from the data
    data = pd.DataFrame(
        {
            "Gate": gate,
            "Group": groups,
            "Positive": positives,
            f"{gene_plot} Expression": gene_expression,
        }
    )

    # Ensure that 'Gate' is treated as a categorical variable with the specified order
    data["Gate"] = pd.Categorical(data["Gate"], categories=gate_order, ordered=True)

    # Calculate the fraction of positive cells within each group and gate
    fraction_positive = data.groupby(["Gate", "Group"])["Positive"].mean().unstack()

    # Calculate SEM for the fraction of positive cells
    sem_positive = data.groupby(["Gate", "Group"])["Positive"].apply(sem).unstack()

    # Plot the bar plot with SEM error bars for the fraction of positive cells
    ax = fraction_positive.plot(
        kind="bar", yerr=sem_positive, capsize=4, figsize=(10, 7), alpha=0.7
    )

    plt.title(f"Fraction of {gene_postive} Positive Cells Across Gates and Groups")
    plt.ylabel(f"Fraction of {gene_postive} Positive Cells")
    plt.xlabel("Gate")
    plt.xticks(rotation=0)
    plt.legend(title="Group")
    # plt.savefig(os.path.join(savdir, gene_postive + '_colored_by_' + gene_plot, f'{gene_postive}_colored_by_{gene_plot}_fraction_positive_across_gates_groups.pdf'))
    plt.show()

In [ ]:
plot_fraction_across_gates_positive(adata, "Itgae", "Klf2", savedir)